# Using VGG-16 as a base model for transfer learning

Now that I've trained a simple model using VGG-16 as a base, using only one Dense layer before output, I'd like to iterate on it to see how I can improve it. I'll be doing the same for InceptionV3 and comparing the two in order to make a final decision for my best model.

In [2]:
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.layers import Dense, Flatten, Dropout
from keras.applications.vgg16 import VGG16
from pickle import dump

from functions import *

%load_ext autoreload
%autoreload 2

### Load data

In [7]:
# Image folder for training
train_dir = 'input_images/full_combined'
val_dir = 'input_images/validation'

# Delete metadata files created by Mac OS
!find . -name ".DS_Store" -delete

In [9]:
# Make generators -- NOW USING SEPARATE VALIDATION SET/FOLDER (see split-folders.ipynb for details)
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   horizontal_flip=True,
                                   rotation_range=20, 
                                   brightness_range=[0.5, 1.5], 
                                   zoom_range=.2)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir, class_mode='binary')
val_gen = val_datagen.flow_from_directory(val_dir, class_mode='binary', shuffle=False)

Found 1619 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


### Instantiate base model

I'll use the VGG-16 model without its original Dense layers for feature extraction. I'll then add my own Dense layers and sigmoid activation output.

In [10]:
# Get base model
vgg_base_model = VGG16(weights='imagenet', 
                       include_top=False,
                       input_shape=(256, 256, 3))

# Freeze layers
for layer in vgg_base_model.layers:
    layer.trainable = False
    
# Check architecture
vgg_base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

## Iterations

Before, I simply added one Dense(512) layer on top of the base architecture. The original architecture of VGG-16 has 3 fully dense layers with 4096 nodes, generating predictions for 1000 classes w/Softmax activation. I'll be stepping that way down, plus only generating binary predictions with sigmoid activation.

I'll use `train_gen` and `val_gen` as defined above, but also bump up the number of epochs to 30 since it seemed like there was still room for improvement.

### Top with 2 Dense(512) layers 

In [12]:
# Instantiate model
vgg_1 = models.Sequential()

# Add base model
vgg_1.add(vgg_base_model)

# Flatten the output layer to 1 dimension
vgg_1.add(Flatten())

# Add 2 fully connected layers with 512 hidden units and ReLU activation
vgg_1.add(Dense(512, activation='relu'))
vgg_1.add(Dense(512, activation='relu'))

# Add a final sigmoid layer with 1 node for classification output
vgg_1.add(Dense(1, activation='sigmoid'))

# Compile
vgg_1.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc', 'Recall', 'Precision'])

In [ ]:
# Train the model
vgg_1_history = vgg_1.fit(train_gen,
                          epochs=30,
                          validation_data=val_gen)

Epoch 1/30
51/51 [==============================] - 329s 6s/step - loss: 1.0882 - acc: 0.7023 - recall: 0.7133 - precision: 0.7081 - val_loss: 0.3027 - val_acc: 0.8700 - val_recall: 0.7400 - val_precision: 1.0000
Epoch 2/30
51/51 [==============================] - 319s 6s/step - loss: 0.3570 - acc: 0.8382 - recall: 0.8193 - precision: 0.8586 - val_loss: 0.2465 - val_acc: 0.8900 - val_recall: 0.9000 - val_precision: 0.8824
Epoch 3/30
51/51 [==============================] - 313s 6s/step - loss: 0.3487 - acc: 0.8474 - recall: 0.8325 - precision: 0.8648 - val_loss: 0.2148 - val_acc: 0.9100 - val_recall: 0.9000 - val_precision: 0.9184
Epoch 4/30
13/51 [======>.......................] - ETA: 3:27 - loss: 0.3212 - acc: 0.8462 - recall: 0.8614 - precision: 0.8365

In [ ]:
vgg_1.save('models/vgg_1.h5')

In [ ]:
visualize_results(vgg_1_history, vgg_1, train_gen, val_gen)

In [ ]:
dump(vgg_1_history.history, open('models/vgg_1_history.pkl', 'wb'))

### 2 Dense layers and aggressive (0.5) Dropout to avoid overfitting

In [ ]:
# Instantiate model
vgg_2 = models.Sequential()

# Add base model
vgg_2.add(vgg_base_model)

# Flatten the output layer to 1 dimension
vgg_2.add(Flatten())

# Add 2 fully connected layers with 512 hidden units and ReLU activation,
# plus Dropout layers
vgg_2.add(Dense(512, activation='relu'))
vgg_2.add(Dropout(0.5))
vgg_2.add(Dense(512, activation='relu'))
vgg_2.add(Dropout(0.5))

# Add a final sigmoid layer with 1 node for classification output
vgg_2.add(Dense(1, activation='sigmoid'))

# Compile
vgg_2.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc', 'Recall', 'Precision'])

In [ ]:
# Train the model
vgg_2_history = vgg_2.fit(train_gen,
                          epochs=30,
                          validation_data=val_gen)

In [ ]:
vgg_2.save('models/vgg_2.h5')

In [ ]:
visualize_results(vgg_2_history, vgg_2, train_gen, val_gen)

In [ ]:
dump(vgg_2_history.history, open('models/vgg_2_history.pkl', 'wb'))

### More Dense layers with decreasing number of nodes

In [ ]:
# Instantiate model
vgg_3 = models.Sequential()

# Add base model
vgg_3.add(vgg_base_model)

# Flatten the output layer to 1 dimension
vgg_3.add(Flatten())

# Add 2 fully connected layers with 512 hidden units and ReLU activation,
# plus Dropout layers
vgg_3.add(Dense(512, activation='relu'))
vgg_3.add(Dense(128, activation='relu'))
vgg_3.add(Dense(64, activation='relu'))


# Add a final sigmoid layer with 1 node for classification output
vgg_3.add(Dense(1, activation='sigmoid'))

# Compile
vgg_3.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc', 'Recall', 'Precision'])